In [5]:
'''整合完成'''
import requests
from bs4 import BeautifulSoup
import urllib.request as req
import json
import pandas as pd
import re

'''關鍵字搜尋'''
search = '資料工程師' 

'''頁數起始值'''
page = 1

'''先建立一個有字串的list，以便進入if迴圈'''
list=['a']

'''建立dataframe的headers'''
columns = [
    '職稱', '公司', '職務類別', '工作內容', '薪資', '公司區域', '詳細地址', '相對位置', '上班時段', '休假制度', '工作經歷', '學歷要求', '語文條件', '語文能力', '擅長工具', '其他條件', '法定福利', '其他福利', '特殊福利'
                ]

df = pd.DataFrame(columns = columns)

while True:   
    if len(list) != 0:
        url = 'https://www.104.com.tw/jobs/search/?ro=0&keyword='+ search +'&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=12&asc=0&page='+ str(page) +'&mode=s&jobsource=2018indexpoc'

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
        }

        res = requests.get(url, headers=headers)
        html = res.text

        soup = BeautifulSoup(html, 'html.parser')

        content = soup.select('div#js-job-content')[0]
        list = content.select('article[class="b-block--top-bord job-list-item b-clearfix js-job-item"]')
#         print(f'\npage:{page}')
        
        for i in list:
            b_tit = i.select('h2[class="b-tit"]')[0]
            for link in b_tit.select('a'):
                link_url = link.get('href')
#                 print(link_url)
                headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
                }

                res = requests.get(str('http:')+link_url, headers=headers)
                html = res.text

                soup = BeautifulSoup(html, 'html.parser')
                key = link_url[21:26]
#                 print(key)
                url = 'https://www.104.com.tw/job/ajax/content/'+str(key)
                headers = {
                'Referer': 'https://www.104.com.tw/job/'+str(key)+'?jobsource=jolist_d_relevance',
                'Content-Type': 'application/json;charset=utf-8',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
                }

                res = requests.get(url, headers=headers)
                
                #宣告一個變數來接住res.text
                jsonData = json.loads(res.text) #轉換成一個list的json字串 
                '''職稱'''
                jobName = jsonData["data"]["header"]["jobName"]
                
                '''公司名稱'''
                comName = jsonData["data"]["header"]["custName"] 
                
                '''職務類別'''
                jC = jsonData["data"]["jobDetail"]["jobCategory"]
                if jC == []:
                    jobCategory = '不拘'
                else:
                    j_C = jC[0]
                    jCC = [j_C['description'] for j in jC]
                    jobCategory = ",".join(jCC) 
                
                
                '''工作內容'''
                jd = jsonData["data"]["jobDetail"]["jobDescription"] 
                j = re.sub(r'[=-]', '*', jd)
                jobDetail = j.replace("\n","")
                
                '''薪資'''
                salary = jsonData["data"]["jobDetail"]["salary"]
                
                '''公司區域'''
                addressRegion = jsonData["data"]["jobDetail"]["addressRegion"]
                
                '''詳細地址'''
                aD = jsonData["data"]["jobDetail"]["addressDetail"]
                addressDetail = '未填寫' if aD == "" else jsonData["data"]["jobDetail"]["addressDetail"]

                '''相對位置'''
                lm = jsonData["data"]["jobDetail"]["landmark"]
                landmark = '未填寫' if lm == "" else jsonData["data"]["jobDetail"]["landmark"]

                '''上班時段'''
                wP = jsonData["data"]["jobDetail"]["workPeriod"] 
                workPeriod = '未填寫' if wP == "" else jsonData["data"]["jobDetail"]["workPeriod"]

                '''休假制度'''
                vP = jsonData["data"]["jobDetail"]["vacationPolicy"]
                vacPolicy = '未填寫' if vP == "" else jsonData["data"]["jobDetail"]["vacationPolicy"] 
                
                '''工作經驗'''
                workExp = jsonData["data"]["condition"]["workExp"]
                
                '''教育程度'''
                edu = jsonData["data"]["condition"]["edu"]
                
                
                '''語言'''
                lan = jsonData["data"]["condition"]["language"]
                if lan == []:
                    language = '不拘'
                    lanAbility = '不拘'
                else:
                    language = jsonData["data"]["condition"]["language"][0]["language"] 
                    lanAbility = jsonData["data"]["condition"]["language"][0]["ability"] #語言能力

                '''擅長工具'''
                s = jsonData["data"]["condition"]["specialty"]
                l = []
                if s == []:
                    spe = '不拘'
                else:
                #     ss = [i['description'] for i in s]
                #     spe = ",".join(ss) 
                    for i in s:
                        ss = i['description']
                        if ss == None:
                            pass
                        else:
                            l.append(ss)
                            spe = ",".join(l) 

                '''其他條件'''
                o = jsonData["data"]["condition"]["other"] 
                if o == "" or o == " " :
                    othCon = '不拘'
                else:    
                    ot = re.sub(r'[=-]', '*', o)
                    othCon = ot.replace("\n","")

                '''法定福利'''
                wl = jsonData["data"]["welfare"]["legalTag"]
                welfare_legal = '未填寫' if wl == [] else ",".join(wl)
 

                '''其他福利'''
                ow = jsonData["data"]["welfare"]["tag"]
                welfare_oth = '未填寫' if ow == [] else ",".join(ow)

                '''自訂福利'''
                w = jsonData["data"]["welfare"]["welfare"]
                if w == "" or w == " " :
                    welfare = '未填寫'
                else:    
                    wf = re.sub(r'[=-]', '*', w)
                    welfare = wf.replace("\n","")

                data = [
                    [jobName, comName, jobCategory, jobDetail, salary, addressRegion, addressDetail, landmark, workPeriod, vacPolicy, workExp, edu, language, lanAbility, spe, othCon, welfare_legal, welfare_oth, welfare]
                ]
                df_crawler = pd.DataFrame(data=data,columns=columns)
                df = df_crawler.append(df)
        print(f'第{page}頁已爬取完成')
        page+=1

    if len(list) == 0:
        break  
        

df = df.reset_index(drop=True)
df = df.sort_index(ascending=False)
df = df.reset_index(drop=True)
df.index = df.index + 1
df.to_csv('C:\\Users\\Drumstick\\Desktop\\104_crawler.csv', index=1, encoding='utf-8-sig', header=1)
print('已爬取完成')

第1頁已爬取完成
第2頁已爬取完成
第3頁已爬取完成
第4頁已爬取完成
第5頁已爬取完成
第6頁已爬取完成
第7頁已爬取完成
第8頁已爬取完成
第9頁已爬取完成
第10頁已爬取完成
第11頁已爬取完成
第12頁已爬取完成
第13頁已爬取完成
第14頁已爬取完成
第15頁已爬取完成
第16頁已爬取完成
第17頁已爬取完成
第18頁已爬取完成
第19頁已爬取完成
第20頁已爬取完成
第21頁已爬取完成
第22頁已爬取完成
第23頁已爬取完成
第24頁已爬取完成
第25頁已爬取完成
第26頁已爬取完成
第27頁已爬取完成
第28頁已爬取完成
第29頁已爬取完成
第30頁已爬取完成
第31頁已爬取完成
第32頁已爬取完成
第33頁已爬取完成
第34頁已爬取完成
第35頁已爬取完成
第36頁已爬取完成
第37頁已爬取完成
第38頁已爬取完成
第39頁已爬取完成
第40頁已爬取完成
第41頁已爬取完成
第42頁已爬取完成
第43頁已爬取完成
第44頁已爬取完成
第45頁已爬取完成
第46頁已爬取完成
第47頁已爬取完成
第48頁已爬取完成
第49頁已爬取完成
第50頁已爬取完成
第51頁已爬取完成
第52頁已爬取完成
第53頁已爬取完成
第54頁已爬取完成
第55頁已爬取完成
第56頁已爬取完成
第57頁已爬取完成
第58頁已爬取完成
第59頁已爬取完成
第60頁已爬取完成
第61頁已爬取完成
第62頁已爬取完成
第63頁已爬取完成
第64頁已爬取完成
第65頁已爬取完成
第66頁已爬取完成
第67頁已爬取完成
第68頁已爬取完成
第69頁已爬取完成
第70頁已爬取完成
第71頁已爬取完成
第72頁已爬取完成
第73頁已爬取完成
第74頁已爬取完成
第75頁已爬取完成
第76頁已爬取完成
第77頁已爬取完成
第78頁已爬取完成
第79頁已爬取完成
第80頁已爬取完成
第81頁已爬取完成
第82頁已爬取完成
第83頁已爬取完成
第84頁已爬取完成
第85頁已爬取完成
第86頁已爬取完成
第87頁已爬取完成
第88頁已爬取完成
第89頁已爬取完成
第90頁已爬取完成
第91頁已爬取完成
第92頁已爬取完成
第93頁已爬取完成
第94頁已爬取完成
第95頁已爬取完成
第96頁已爬取完成
第97頁已爬取完成
第98頁已爬取完成
第99頁已爬取完成
第100頁已爬取完成
第101頁已爬取